# Chapter 5: Responsible Prompting

---

**Lesson:**

While there is enormous potential with Generative AI models, we must always take into account the inherent risks of these models, particularly as it pertains to the effect on people / groups of people.

In [ ]:
%pip install ipython

import json
import boto3
from PIL import Image
import base64
import io
from random import randint
from IPython.display import display

def generate_image(text):
    # Create a new BedrockRuntime client.
    bedrock_runtime = boto3.client(
        "bedrock-runtime",
        region_name="us-east-1",
    )
    
    # Configure the inference parameters.
    inference_params = {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": text,
            # "negativeText": negative_text,
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "quality": "premium",
            "height": 1024,
            "width": 1024,
            "cfgScale": 8.0,
            "seed": randint(0, 3000),  # Use a random seed
        },
    }
    
    # Invoke the model.
    response = bedrock_runtime.invoke_model(
        modelId="amazon.titan-image-generator-v1", body=json.dumps(inference_params)
    )
    
    # Convert the JSON string to a dictionary.
    response_body = json.loads(response["body"].read())
    
    # Loop through the generated images and display each in the notebook.
    images = response_body["images"]
    for i in range(len(images)):
        image_data = images[i]
        image_bytes = base64.b64decode(image_data)
        image = Image.open(io.BytesIO(image_bytes))
        display(image)

## The inherent risk of our models

Amazon Titan Image Generator G1 and all other image generation models are subject to some level of risk, whether it is due to malicious or non-malicious intent. Let's discuss two of the most pertinent topics for these models.

### Malicious image production

Not everyone has positive intent when developing images via Amazon Titan Image Generator G1. Misuse of models can include efforts to defame or dehumanize people or groups of people. These efforts can range from ad hominem attacks in a community to political statements (particulalrly during elections) and attacks on cultures and beliefs. Model providers and the wider community continue to innovate to prevent the intentional misuse of models. However, it is very important to keep in mind the potential for misuse both in models you deploy as well as for images you see day to day that may have been modified to demean a person or people. As a user of Amazon Titan Image Generator G1, continually keep up to date with any "holes" in the model's ability to detect and deny malicious intent, and work to prevent misuse by any users of your application.

### Attribute association

An unfortunate side effect of many models is assumed correlation of certain attributes (such as eye and hair color) with other attributes (such as race). While this attribute association represents non-malicious intent by the model, the effect is similar to the above - poor representation of certain segments of the population. What's more is this issue can spill into other areas beyond race, culture, etc. as our models associate keywords incorrectly and generate undesired images (e.g., sustainability is most often associated with green technologies / ways of living, but this term can also simply designate a long-term, strategic focus). With this in mind, it is important to pay attention to the keywords you use and understand if your model will incorrectly assume what you are looking for with the word(s) in your prompt.

Let's explore this concept in a couple of examples.

**Example 5.1 - Blue eyes**

While there are certainly populations that predominantly have blue eyes, this eye color is not limited solely to those groups of people. However, as Amazon Titan Image Generator G1 makes decisions based on probabilities, the model will often generate based on what it has "seen" most often in its original, training corpus of online data.

In [ ]:
text = "a person with blue eyes"
generate_image(text)

**Example 5.2 - What is football?**

Let's see if Amazon Titan Image Generator G1 recognizes football can mean different things depending on where you are in the world.

In [ ]:
text = "football"
generate_image(text)

One of the best fixes to the above issues is adding more keywords, clarifying what you are looking for. However, it is very important to understand the inherent biases beneath the surface of your models, at the very least, so you can understand why you may not be getting the results you desire and how you can modify your prompts to produce more accurate results.

# Chapter 5 - END